In [ ]:
# stdlib

# stdlib

# third party
import veilid
from veilid import KeyPair

In [ ]:
host = "localhost"
port = 5959

In [ ]:
async def noop_callback(*args, **kwargs):
    return


async def connect(host: str, port: int):
    conn = await veilid.json_api_connect(host, port, noop_callback)
    return conn

In [ ]:
conn = await connect(host, port)

In [ ]:
# Creating a new routing context
router = await (await conn.new_routing_context()).with_default_safety()

In [ ]:
router

In [ ]:
alice_record = await router.create_dht_record(veilid.DHTSchema.dflt(1))

In [ ]:
# Creation of a Record in DHT DFLT schema , creates a new public and private key pair for the owner
# that is different from the NodeID public key
alice_private_key = alice_record.owner_secret
alice_public_key = alice_record.owner

In [ ]:
alice_private_key, alice_public_key, alice_record.key

In [ ]:
# Close the record
await router.close_dht_record(alice_record.key)

In [ ]:
key_pair = KeyPair.from_parts(key=alice_public_key, secret=alice_private_key)

In [ ]:
key_pair

In [ ]:
record_open = await router.open_dht_record(alice_record.key, key_pair)

In [ ]:
record_open.key

In [ ]:
await router.set_dht_value(record_open.key, 0, b"Hello World")

In [ ]:
type(record_open.key)